# Языковые модели

Обучим символьные модели для генерации имён динозавров.

### Данные: имена динозавров

In [1]:
!wget https://raw.githubusercontent.com/artemovae/ML-for-compling/master/2018/dinos.txt

--2020-04-01 17:49:05--  https://raw.githubusercontent.com/artemovae/ML-for-compling/master/2018/dinos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19909 (19K) [text/plain]
Saving to: ‘dinos.txt’

dinos.txt           100%[===================>]  19.44K  --.-KB/s    in 0.007s  

2020-04-01 17:49:05 (2.78 MB/s) - ‘dinos.txt’ saved [19909/19909]



In [3]:
!cat dinos.txt | wc -l

1535


## Марковская цепь


Снижаем регистр, добавляем символы начала и конца строки.

In [4]:
names = ['<' + name.strip().lower() + '>' for name in open('dinos.txt').readlines()]
names[:10]

['<aachenosaurus>',
 '<aardonyx>',
 '<abdallahsaurus>',
 '<abelisaurus>',
 '<abrictosaurus>',
 '<abrosaurus>',
 '<abydosaurus>',
 '<acanthopholis>',
 '<achelousaurus>',
 '<acheroraptor>']

In [0]:
import nltk

Вычислим частоту каждого символа в корпусе имен динозавров:

In [0]:
chars = [char for name in names for char in name]


In [0]:
freq = nltk.FreqDist(chars)

In [9]:
freq

FreqDist({'<': 1536,
          '>': 1536,
          'a': 2487,
          'b': 171,
          'c': 539,
          'd': 341,
          'e': 913,
          'f': 37,
          'g': 360,
          'h': 548,
          'i': 944,
          'j': 55,
          'k': 141,
          'l': 617,
          'm': 328,
          'n': 1081,
          'o': 1710,
          'p': 552,
          'q': 23,
          'r': 1704,
          's': 2285,
          't': 852,
          'u': 2123,
          'v': 111,
          'w': 41,
          'x': 85,
          'y': 266,
          'z': 60})

In [0]:
print(list(freq.keys()))

['j', 'u', 'g', 'n', 'y', 'r', 'l', 'o', 's', 'v', 'c', 'z', 'i', 'k', 'q', 'b', '>', 'd', '<', 'p', 't', 'm', 'w', 'a', 'f', 'x', 'e', 'h']


In [10]:
freq.most_common(10)

[('a', 2487),
 ('s', 2285),
 ('u', 2123),
 ('o', 1710),
 ('r', 1704),
 ('<', 1536),
 ('>', 1536),
 ('n', 1081),
 ('i', 944),
 ('e', 913)]

Определим функцию, чтобы оценить веротность символа:

In [0]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l

In [13]:
print('p(a) = %1.4f' %unigram_prob('s'))

p(a) = 0.1065


### Биграммная модель

Вычислим условную вероятность каждого символа в зависимости от того, какой символ стоял на предыдущей позиции.

In [0]:
bigrams = nltk.bigrams(chars) # список пар идущих друг за другом символов

In [0]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))

Оценим условные вероятности с помощью MLE.

In [0]:
# просто подсчитавыем, сколько раз какой символ встретился после каждого из символов
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [39]:
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0044
p(a b) = 0.0097
p(a u) = 0.3181


In [51]:
cprob['a'].generate() # просим породить следующий символ

'n'

### Задание 1

Напишите функцию, которая генерирует имя динозавра **фиксированной** длины. Используйте '<' как начальный символ.

In [0]:
def generate_fixed_length(n):
  name = '<'
  while len(name) < n:
    name += cprob[name[-1]].generate()
  name += '>'
  return name

In [65]:
generate_fixed_length(10)

'<hanyran><>'

### Задание 2

1) Напишите функцию, которая генерирует имя динозавра любой дины.

2) сделайте возможность при вызове функции самостоятельно подавать начало имени динозавра.

In [0]:
def generate_start(start):
  name = '<'+ start
  while name[-1] != '>':
    name += cprob[name[-1]].generate()
    if len(name) > 10 and name[-1] != '>':
      name += '>'
  return name 

In [102]:
generate_start('')

'<liskenga>'

### Задание 3

Напишите функцию, которая принимает имя динозавра (строку) и возвращает вероятность этого имени на основании символов, из которых оно состоит.  

In [0]:
def estimate_prob(dino_name):
  total_proba = 1
  pairs = nltk.bigrams(dino_name)
  for first_sym, last_sym in pairs:
    current_proba = cprob[first_sym].prob(last_sym)
    total_proba *= current_proba
  return total_proba

In [103]:
estimate_prob('<liskenga>')

1.1626889613524152e-10

In [105]:
estimate_prob('<llllllll>')

3.2498263695790903e-13

### Задание 4

Напишите функцию, которая принимает два имени динозавра и возвращает из них самое вероятное.

In [0]:
def choose_more_probable(word1, word2):
  if estimate_prob(word1) > estimate_prob(word2):
    return word1
  return word2

In [112]:
choose_more_probable('<apatodon>', '<liskenga>')

'<apatodon>'

In [110]:
[name for name in names if len(name) == 10][:5]

['<aardonyx>', '<aliwalia>', '<alocodon>', '<anthodon>', '<apatodon>']

### Панграмы

Давайте напишем функцию, которая генерирует панграмы -- имена динозавров, не содержащие одинаковых букв.

In [0]:
def generate_pangram():
  name = '<'
  while name[-1] != '>':

    # 30 раз пытемся сгенерировать новую букву
    for i in range(30):
        new_symbol = cprob[name[-1]].generate()
        if new_symbol not in name: # если получилось, добавляем её и выходим из цикла
            name += new_symbol
            break
    else:
        print('no new symbol generated in 30 attempts')
        name += '>'

    if len(name) > 30:
        name += '>'
  return name

In [131]:
generate_pangram()

no new symbol generated in 30 attempts


'<aurypinochelb>'

## На уровне слов

Давайте теперь возьмём много текстов -- например, [книжку по программированию в txt](https://drive.google.com/open?id=17oT5ZEmUcSS_C_0eS6jZBMhn-q2YipaK) -- и попробуем сделать то же самое, но со словами.

In [132]:
from google.colab import files
uploaded = files.upload()

Saving coding.txt to coding.txt


In [0]:
from nltk import word_tokenize, sent_tokenize

In [0]:
with open('coding.txt', encoding='utf-8') as f:
    text = f.read()

In [143]:
print(text[:200])

 Кормен (Renee Connen).

Предисловие
Как компьютеры решают задачи? Как ваш маленький GPS в считанные секунды на­
ходит самый быстрый пуrь из несметного множества возможных маршруrов? Когда вы
покупае


In [138]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
sentences = sent_tokenize(text)

In [141]:
sentences[23:25]

['Возможно, вы уrолите голод одними са\xad\nлатиками.',
 'А может быть, вам так понравится, что вы закажете официанту обильный обед и\nс нетерпением будете его ждать.']

In [0]:
def preprocess_sentences(sentences):
  return ['bos ' + sent.lower() + ' eos' for sent in sentences]

In [0]:
preprocessed = preprocess_sentences(sentences)

In [0]:
tokenized_text = [token for sent in preprocessed for token in word_tokenize(sent)]

In [0]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(tokenized_text))

In [155]:
cfreq['рекурсия']

FreqDist({'*/': 1,
          '34': 1,
          'завершается': 1,
          'заканчивается': 1,
          'исчерпается': 1,
          'раб': 1,
          'работала': 1,
          'с': 1})

In [0]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [157]:
cprob['рекурсия'].prob('завершается')

0.125

In [0]:
def generate():
  sent = ['bos']
  while sent[-1] != 'eos':
    sent.append(cprob[sent[-1]].generate())
    if len(sent) > 30 and sent[-1] != 'eos':
      sent.append('eos')
  return ' '.join(sent) 

In [169]:
print(generate())

bos эта функция fchdir и putenv xsi — не совсем :  значения для всех трех списков на другой поток управления выводом без управляющего терминала ( stat = 11о и для eos


### RNN на keras

Если осталось время.

Не очень закоменченная, сложная и не обязательная для понимания, но любопытная часть про RNN: https://colab.research.google.com/drive/1tJ5qM6a1EP8dBlxE6KuN8DXLjFUFxNIz.